# Linear Support Vector Machine (SVM)

We've now seen how to optimise analytic functions using PyTorch's optimisers, and in the previous labs and exercises we played with training simple machine learning models with hand-coded gradient descent. Let's put everything together and implement a Soft-Margin Linear Support Vector Machine, which we'll train on some artifically generated data using a range of optimisers.

In [25]:
# We're going to use a library called celluloid to make animations that work on colab
try: 
    from celluloid import Camera
except:
    !pip install celluloid

from IPython.display import HTML
import torch
import torch.optim as optim

In [2]:
torch.cuda.is_available()

False

## SVM Recap 合页损失函数，软间隔支持向量机

Recall that an SVM tries to find the maximum margin hyperplane which separates the data classes. For a soft margin SVM
where $\textbf{x}$ is our data, we minimize:

\begin{equation}
\left[\frac 1 n \sum_{i=1}^n \max\left(0, 1 - y_i(\textbf{w}\cdot \textbf{x}_i - b)\right) \right] + \lambda\lVert \textbf{w} \rVert^2
\end{equation}

We can formulate this as an optimization over our weights $\textbf{w}$ and bias $b$, where we minimize the
hinge loss subject to a level 2 weight decay term. The hinge loss for some model outputs
$z = \textbf{w}\textbf{x} + b$ with targets $y$ is given by:

\begin{equation}
\ell(y,z) = \max\left(0, 1 - yz \right)
\end{equation}

First, complete the following function to implement the hinge loss for batches of predictions `y_pred` and targets `y_true`. You should return the mean of the hinge loss across the batch. Note that this is a binary problem with labels are chosen to be $\{-1,1\}$.

In [7]:
def hinge_loss(y_pred, y_true):
    return max(0, 1 - y_pred * y_true)

## Defining the SVM

Defining the SVM is pretty simple - it's just a basic linear classifier like a Perceptron; what distinguishes it is the loss.  We'll wrap it up in a function:

In [8]:
def svm(x, w, b):
    h = (w*x).sum(1) + b
    return h

Creating Synthetic Data
-----------------------------------------------

Now for some data, 1024 samples should do the trick. We normalise here so that our random init is in the same space as
the data:

In [11]:
import numpy as np
from sklearn.datasets._samples_generator import make_blobs

X, Y = make_blobs(n_samples=1024, centers=2, cluster_std=1.2, random_state=1)
X = (X - X.mean()) / X.std()
Y[np.where(Y == 0)] = -1
X, Y = torch.FloatTensor(X), torch.FloatTensor(Y)
print(X.size(), Y.size())

torch.Size([1024, 2]) torch.Size([1024])


In [10]:
print(X,Y)

tensor([[ 0.0908,  1.2707],
        [ 0.1552,  1.6161],
        [-1.5526, -0.2490],
        ...,
        [-0.1173,  1.6335],
        [ 0.2541,  1.3813],
        [ 0.3465,  1.8740]]) tensor([-1., -1.,  1.,  ..., -1., -1., -1.])


For the first time, we're going to do proper mini-batch gradient descent. As such, we actually need to be able to produce batches of data. PyTorch has the concept of datasets (which represent entire collections of data) and data loaders (which allow us to iterate batches of data from a dataset). This allows the framework to do all the hard work for us:

In [12]:
from torch.utils import data

dataset = data.TensorDataset(X,Y) # create your datset data.TensorDataset is like a function which could help us zip
#data into dataset class. To implement this, we need to confirm the first dimension of two tensors are the same.
dataloader = data.DataLoader(dataset, batch_size=32, shuffle=True) # create your dataloader

In [18]:
for epoch in range(2):
  for i, data in enumerate(dataloader):
    inputs, labels = data
    print(i, inputs.size(), labels.size())

0 torch.Size([32, 2]) torch.Size([32])
1 torch.Size([32, 2]) torch.Size([32])
2 torch.Size([32, 2]) torch.Size([32])
3 torch.Size([32, 2]) torch.Size([32])
4 torch.Size([32, 2]) torch.Size([32])
5 torch.Size([32, 2]) torch.Size([32])
6 torch.Size([32, 2]) torch.Size([32])
7 torch.Size([32, 2]) torch.Size([32])
8 torch.Size([32, 2]) torch.Size([32])
9 torch.Size([32, 2]) torch.Size([32])
10 torch.Size([32, 2]) torch.Size([32])
11 torch.Size([32, 2]) torch.Size([32])
12 torch.Size([32, 2]) torch.Size([32])
13 torch.Size([32, 2]) torch.Size([32])
14 torch.Size([32, 2]) torch.Size([32])
15 torch.Size([32, 2]) torch.Size([32])
16 torch.Size([32, 2]) torch.Size([32])
17 torch.Size([32, 2]) torch.Size([32])
18 torch.Size([32, 2]) torch.Size([32])
19 torch.Size([32, 2]) torch.Size([32])
20 torch.Size([32, 2]) torch.Size([32])
21 torch.Size([32, 2]) torch.Size([32])
22 torch.Size([32, 2]) torch.Size([32])
23 torch.Size([32, 2]) torch.Size([32])
24 torch.Size([32, 2]) torch.Size([32])
25 torch.S

Visualizing the Training
----------------------------------------

We now aim to create a nice visualisation, such as the one below, that shows what happens as our SVM learns.

![svmgif](https://raw.githubusercontent.com/ecs-vlc/torchbearer/master/docs/_static/img/svm_fit.gif)

The code for the visualisation (using [pyplot](https://matplotlib.org/api/pyplot_api.html)) is a bit ugly but we'll
try to explain it to some degree. First, we need a mesh grid `xy` over the range of our data:

In [21]:
delta = 0.01
x = np.arange(X[:, 0].min(), X[:, 0].max(), delta)
y = np.arange(X[:, 1].min(), X[:, 1].max(), delta)
print(x.shape, y.shape)
x, y = np.meshgrid(x, y)
print(x.shape, y.shape)
xy = list(map(np.ravel, [x, y]))

(296,) (319,)
(319, 296) (319, 296)


In [51]:
len(xy[1])

94424

In [22]:
xy

AttributeError: ignored

Now things get a little strange. We start by evaluating our model over the mesh grid from earlier.

For our outputs $z \in \textbf{Z}$, we can make some observations about the decision boundary. First, that we are
outside the margin if $z \lt -1$ or $z \gt 1$. Conversely, we are inside the margine where $z \gt -1$
or $z \lt 1$. 

This whole process is shown in the function below, which we can call at the end of every epoch. The `camera` takes snapshots of the current plot and is used later to render a video.

In [23]:
import matplotlib
import matplotlib.pyplot as plt

def draw_margin(w, b, camera):
    w = w.data.numpy()
    b = b.data.numpy()

    z = (w.dot(xy) + b).reshape(x.shape)
    z[np.where(z > 1.)] = 4
    z[np.where((z > 0.) & (z <= 1.))] = 3
    z[np.where((z > -1.) & (z <= 0.))] = 2
    z[np.where(z <= -1.)] = 1

    plt.scatter(x=X[:, 0], y=X[:, 1], c="black", s=10)
    plt.contourf(x, y, z, cmap=plt.cm.jet, alpha=0.5)
    camera.snap()

Since we don't know that our data is linearly separable, we would like to use a soft-margin SVM. That is, an SVM for
which the data does not all have to be outside of the margin. This takes the form of a weight decay term,
$\lambda\lVert \textbf{w} \rVert^2$ in the above equation. This term is called weight decay because the gradient
corresponds to subtracting some amount ($2\lambda\textbf{w}$) from our weights at each step. 

Most PyTorch optimisers actually have weight decay built in to them as an option (`weight_decay=...`), so its trivial to incorporate this. 

At this point we are ready to create and train our model. We've written most of the code, but you'll need to implement the forward and backward pass:

In [28]:
# Set up drawing
fig = plt.figure(figsize=(5, 5))
camera = Camera(fig)

w = torch.randn(1, 2, requires_grad=True)
b = torch.randn(1, requires_grad=True)

opt = optim.SGD([w,b], lr=0.1, weight_decay=0.01)

for epoch in range(50):
    for batch in dataloader:
        opt.zero_grad()
        output = 
        opt.step()
    draw_margin(w, b, camera)
    
# create the animation and display it
anim = camera.animate()
plt.close()
HTML(anim.to_html5_video())

[tensor([[ 0.0963,  1.2839],
        [-1.8256, -0.3255],
        [-0.0962,  1.6876],
        [-1.3573, -0.2220],
        [ 0.6248,  1.4822],
        [ 0.4132,  1.1995],
        [-1.0685, -0.0578],
        [-0.0695,  1.4791],
        [-1.3503, -0.0977],
        [ 0.1829,  1.3146],
        [ 0.1521,  1.4411],
        [ 0.1573,  1.7199],
        [-1.2480, -0.2822],
        [-1.3140, -0.1025],
        [-1.2952,  0.0288],
        [ 0.0979,  1.3848],
        [ 0.4805,  1.7405],
        [ 0.4188,  1.2508],
        [-1.2140, -0.5718],
        [-1.4562, -0.5585],
        [-0.9545, -0.3411],
        [ 0.4494,  0.9667],
        [ 0.5480,  1.5074],
        [ 0.3249,  1.7419],
        [-1.2569, -0.2783],
        [-1.2771,  0.1090],
        [-1.3576, -0.5133],
        [-1.4243, -0.4125],
        [ 0.1010,  1.3603],
        [ 0.3619,  1.1907],
        [-1.8249, -0.0862],
        [ 0.4060,  1.5244]]), tensor([-1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1.,
         1., -1., -1., 

<Figure size 360x360 with 0 Axes>

Now do some further experiments. What optimiser and parameters gets you to a good solution the quickest? Do you notice that when the model is near a solution it jitters around upon each step? Can you add some kind of learning rate decay or schedule from the `torch.optim.lr_scheduler` package to reduce the learning rate over time?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()